In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
student_registr = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-k-parfenova-21/Dop/studentRegistration.csv', sep = ',')
student_assessment = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-k-parfenova-21/Dop/studentAssessment.csv', sep = ',')
assessments = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-k-parfenova-21/Dop/assessments.csv', sep = ',')
courses = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-k-parfenova-21/Dop/courses.csv', sep = ',')

**Файлы:**

**assessments.csv** — этот файл содержит информацию об оценках в тесте. Обычно каждый предмет в семестре включает ряд тестов с оценками, за которыми следует заключительный экзаменационный тест (экзамен).

code_module — идентификационный код предмета.

code_presentation — семестр (Идентификационный код).

id_assessment — тест (Идентификационный номер ассессмента).

assessment_type — тип теста. Существуют три типа оценивания: оценка преподавателя (TMA), компьютерная оценка (СМА), экзамен по курсу (Exam).

date — информация об окончательной дате сдачи теста. Рассчитывается как количество дней с момента начала семестра. Дата начала семестра имеет номер 0 (ноль).

weight — вес теста в % в оценке за курс. Обычно экзамены рассматриваются отдельно и имеют вес 100%; сумма всех остальных оценок составляет 100%.

**courses.csv** — файл содержит список предметов по семестрам.

code_module — предмет (идентификационный код).

code_presentation — семестр (идентификационный код).

module_presentation_length — продолжительность семестра в днях.

**studentAssessment.csv** — этот файл содержит результаты тестов студентов. Если учащийся не отправляет работу на оценку, результат не записывается в таблицу.

id_assessment — тест (идентификационный номер).

id_student — идентификационный номер студента.

date_submitted — дата сдачи теста студентом, измеряемая как количество дней с начала семестра.

is_banked — факт перезачета теста с прошлого семестра (иногда курсы перезачитывают студентам, вернувшимся из академического отпуска).

score — оценка учащегося в этом тесте. Диапазон составляет от 0 до 100. Оценка ниже 40 неудачная/неуспешная сдача теста.

**studentRegistration.csv** — этот файл содержит информацию о времени, когда студент зарегистрировался для прохождения курса в семестре.

code_module — предмет (идентификационный код).

code_presentation — семестр (идентификационный код)

id_student — идентификационный номер студента.

date_registration — дата регистрации студента. Это количество дней, измеренное от начала семестра (например, отрицательное значение -30 означает, что студент зарегистрировался на прохождение курса за 30 дней до его начала).

date_unregistration — дата отмены регистрации студента с предмета. У студентов, окончивших курс, это поле остается пустым.

In [3]:
student_assessment.isna().sum()

id_assessment       0
id_student          0
date_submitted      0
is_banked           0
score             173
dtype: int64

У некоторых студентов не стоит оценка, поэтому вместо пропуска - ставим 0, для дальнейших расчетов

In [4]:
student_assessment = student_assessment.fillna(0)

**1. Сколько студентов успешно сдали только один курс? (Успешная сдача — это зачёт по курсу на экзамене)**

In [6]:
x = assessments.query('assessment_type == "Exam"') 
y = student_assessment.query('score >= 40') 

all_exam = x.merge(student_assessment, on = 'id_assessment', how = 'inner') # DF со всеми экзаменами

passed_exam = x.merge(y, on = 'id_assessment', how = 'inner')  # DF со сданными экзаменами

In [7]:
passed_exam.query('is_banked < 0').is_banked.count()

0

Значит пересдач по экзаменам не было

In [9]:
number_of_courses = passed_exam.groupby(['code_module', 'code_presentation', 'id_student'], as_index = False) \
.agg({'score': 'count', 'is_banked': 'sum'})

number_of_courses.head()

,code_module,code_presentation,id_student,score,is_banked
0,CCC,2014B,29764,1,0
1,CCC,2014B,29820,1,0
2,CCC,2014B,40604,1,0
3,CCC,2014B,42638,1,0
4,CCC,2014B,46605,1,0


In [10]:
number_of_courses = number_of_courses.groupby('id_student', as_index = False).agg({'code_module':'count'})
number_of_courses.id_student.count()

4097

In [11]:
number_of_courses.query('code_module == 1').id_student.count()

3802

Ответ: 

   Студентов, сдавших только один курс, *3802*

**2. Выяви самый сложный и самый простой экзамен: найди курсы и экзамены в рамках курса, которые обладают самой низкой
и самой высокой завершаемостью**

завершаемость = кол-во успешных экзаменов / кол-во всех попыток сдать экзамен

In [12]:
exam_complex_1 = passed_exam.groupby(['code_module', 'id_assessment'], as_index = False) \
.agg({'is_banked': 'count'})

exam_complex_2 = all_exam.groupby(['code_module', 'id_assessment'], as_index = False) \
.agg({'is_banked': 'count'})

In [13]:
exam_complex_1['completeness'] = exam_complex_1.is_banked / exam_complex_2.is_banked
exam_complex_1.rename(columns = {'is_banked': 'number_exams_passed'})
completeness = exam_complex_1.round(2)
completeness

,code_module,id_assessment,is_banked,completeness
0,CCC,24290,664,0.89
1,CCC,24299,1019,0.87
2,DDD,25340,504,0.84
3,DDD,25354,878,0.91
4,DDD,25361,485,0.93
5,DDD,25368,842,0.89


In [14]:
max_completeness = completeness.sort_values('completeness', ascending = False).head(1)
max_completeness

,code_module,id_assessment,is_banked,completeness
4,DDD,25361,485,0.93


In [15]:
min_completeness = completeness.sort_values('completeness', ascending = True).head(1)
min_completeness

,code_module,id_assessment,is_banked,completeness
2,DDD,25340,504,0.84


Ответ:

   самый *сложный* экзамен: по курсу "DDD", тест №: 25340

   самый *простой* экзамен: по курсу "DDD", тест №: 25361

**3. По каждому предмету определи средний срок сдачи экзаменов 
(под сдачей понимаем последнее успешное прохождение экзамена студентом)**

In [16]:
date_mean_1 = passed_exam.groupby('code_module', as_index = False) \
.agg({'date_submitted': 'mean'}).round(2)

date_mean_1

,code_module,date_submitted
0,CCC,239.35
1,DDD,237.98


Ответ:

   по курсу "CCC", средний срок сдачи: ~ 240 дней

   по курсу "DDD", средний срок сдачи: ~ 238 дней

In [17]:
date_mean_2 = passed_exam.groupby(['id_assessment', 'code_module'], as_index = False) \
.agg({'date_submitted': 'mean'}).round(2)

date_mean_2

,id_assessment,code_module,date_submitted
0,24290,CCC,231.58
1,24299,CCC,244.40
2,25340,DDD,230.16
3,25354,DDD,239.51
4,25361,DDD,234.94
5,25368,DDD,242.80


Второй вариант. Если считать средний срок сдачи экзаменов(по id), а не курса.

**4. Выяви самые популярные курсы (ТОП-3) по количеству регистраций на них**

In [18]:
top_course = assessments.merge(student_registr, on = ['code_module', 'code_presentation'], how = 'inner') \
.merge(student_assessment, on = ['id_assessment', 'id_student'], how = 'inner') 

In [19]:
top_registration = top_course.query('date_registration != "NaN"') \
.groupby('code_module', as_index = False) \
.agg({'date_registration': 'count'}) \
.sort_values('date_registration', ascending = False).head(3)

top_registration

,code_module,date_registration
5,FFF,54815
1,BBB,43032
3,DDD,30859


**А также курсы с самым большим оттоком (ТОП-3)**

In [20]:
top_unregistration = top_course.query('date_unregistration != "NaN"') \
.groupby('code_module', as_index = False) \
.agg({'date_unregistration': 'count'}) \
.sort_values('date_unregistration', ascending = False).head(3)

top_unregistration

,code_module,date_unregistration
3,DDD,3523
5,FFF,3397
1,BBB,2693


Ответ:

    В топ популярных курсов попали: 
    FFF = 54815 
    BBB = 43032 
    DDD = 30859 
    
    Следовательно, и отток по этим курсам с большей вероятностью будет в топе. Как в итоге и вышло:
    DDD = 3523 
    FFF = 3397 
    BBB = 2693 

Значимость оттока:
    
   отток курса / приток курса

In [21]:
top_relevance = top_registration.merge(top_unregistration, on = 'code_module')
top_relevance['relevance'] = top_relevance.date_unregistration / top_relevance.date_registration
top_relevance.sort_values('relevance', ascending = False)

,code_module,date_registration,date_unregistration,relevance
2,DDD,30859,3523,0.114164
1,BBB,43032,2693,0.062581
0,FFF,54815,3397,0.061972


**5. Напиши функцию на python, позволяющую строить когортный (семестровый) анализ. 
В период с начала 2013 по конец 2014 выяви семестр с самой низкой завершаемостью курсов и самыми долгими средними сроками сдачи курсов**

In [22]:
def cohort_analysis(x):
    
    exam_finality_1 = passed_exam.groupby(['code_presentation', 'code_module', 'id_assessment'], as_index = False) \
    .agg({'is_banked': 'count', 'date_submitted': 'mean'})
    
    exam_finality_2 = all_exam.groupby(['code_presentation', 'code_module', 'id_assessment'], as_index = False) \
    .agg({'is_banked': 'count'})
    
    exam_final = exam_finality_1.rename(columns = {'is_banked': 'number_exams_passed'})
    
    exam_final['completeness'] = exam_final.number_exams_passed / exam_finality_2.is_banked
    
    exam_final = exam_final.round(2)
    
    n = exam_final.completeness.min()
    nl = exam_final.query('completeness == @n')
    nl = nl.code_presentation.min()
    k = exam_final.date_submitted.max()
    kl = exam_final.query('date_submitted == @k')
    kl = kl.code_presentation.min()
            
    print('Семестр с самой низкой завершаемостью курсов:', nl,'=', n)
    print('Семестр с самыми долгими средними сроками сдачи курсов:', kl,'=', k)
    
    print('Информация по семестру', x)
    
    info = exam_final[['code_presentation', 'code_module', 'id_assessment', 'date_submitted', 'completeness']].query('code_presentation == @x')
    print(info)

In [23]:
cohort_analysis('2013B') # 2013B - взят как пример. Чтобы показать, какие данные выводятся

Семестр с самой низкой завершаемостью курсов: 2013B = 0.84
Семестр с самыми долгими средними сроками сдачи курсов: 2014J = 244.4
Информация по семестру 2013B
  code_presentation code_module  id_assessment  date_submitted  completeness
0             2013B         DDD          25340          230.16          0.84


Ответ:

  Семестр с самой низкой завершаемостью курсов: 2013B = 0.84
  
  
  Семестр с самыми долгими средними сроками сдачи курсов: 2014J = 244.4  
    

**6. Часто для качественного анализа аудитории использую подходы, основанные на сегментации. Используя python, построй адаптированные RFM-кластеры студентов, чтобы качественно оценить свою аудиторию. В адаптированной кластеризации можешь выбрать следующие метрики: R - среднее время сдачи одного экзамена, F - завершаемость курсов, M - среднее количество баллов, получаемое за экзамен. Подробно опиши, как ты создавал кластеры. Для каждого RFM-сегмента построй границы метрик recency, frequency и monetary для интерпретации этих кластеров.**

In [28]:
exam_1 = passed_exam.groupby(['code_module', 'id_student'], as_index = False) \
.agg({'score': 'max', 'date_submitted': 'mean'})

exam_1['completeness'] = exam_1.score / 100

rfm_analysis = exam_1.rename(columns = {'date_submitted': 'R',
                    'score': 'M', 
                    'completeness': 'F'}) 

rfm_analysis = rfm_analysis[['code_module', 'id_student', 'R', 'F', 'M']]

rfm_analysis = rfm_analysis.round(2)
rfm_analysis

,code_module,id_student,R,F,M
0,CCC,23698,243.0,0.80,80.0
1,CCC,27116,243.0,0.96,96.0
2,CCC,28787,243.0,0.44,44.0
3,CCC,29420,244.0,0.62,62.0
4,CCC,29639,248.0,0.74,74.0
...,...,...,...,...,...
4384,DDD,2693243,230.0,0.82,82.0
4385,DDD,2694886,236.0,0.69,69.0
4386,DDD,2694933,230.0,0.73,73.0
4387,DDD,2695608,237.0,0.73,73.0


In [29]:
def finish(x):
    if x > 0.9 and x <= 1:
         return '5.0'
    elif x > 0.8 and x <= 0.9:
        return '4.5'
    elif x > 0.7 and x <= 0.8:
        return '4.0'
    elif x > 0.6 and x <= 0.7:
        return '3.5'
    elif x > 0.5 and x <= 0.6:
        return '3.0'
    else:
        return '2.0'  

In [30]:
rfm_analysis['F'] = rfm_analysis['F'].apply(lambda x: finish(x))
rfm_analysis = rfm_analysis.astype({'F': 'float'})
rfm_analysis

,code_module,id_student,R,F,M
0,CCC,23698,243.0,4.0,80.0
1,CCC,27116,243.0,5.0,96.0
2,CCC,28787,243.0,2.0,44.0
3,CCC,29420,244.0,3.5,62.0
4,CCC,29639,248.0,4.0,74.0
...,...,...,...,...,...
4384,DDD,2693243,230.0,4.5,82.0
4385,DDD,2694886,236.0,3.5,69.0
4386,DDD,2694933,230.0,4.0,73.0
4387,DDD,2695608,237.0,4.0,73.0


In [31]:
quantiles = rfm_analysis.quantile(q=[0.25, 0.5, 0.75]) # 1
quantiles = quantiles.to_dict()

def RFM(x,p,d):  # 2
    if x >= d[p][0.75]:
        return 1
    elif x >= d[p][0.5]:
         return 2
    elif x >= d[p][0.25]:
         return 3
    else:
         return 4

rfm_analysis['r_quartile'] = rfm_analysis['R'].apply(RFM, args=('R', quantiles))  # 3
rfm_analysis['f_quartile'] = rfm_analysis['F'].apply(RFM, args=('F', quantiles))
rfm_analysis['m_quartile'] = rfm_analysis['M'].apply(RFM, args=('M', quantiles))

rfm_analysis['RFM'] = rfm_analysis.r_quartile.map(str) + rfm_analysis.f_quartile.map(str) + rfm_analysis.m_quartile.map(str) #4

1. Данные были разделение на 4 сегмента. По квантилям на равные части.
2. Функция, считывает данные по столбцам и индексам и распределяет по сегментам.
3. Создание нового столбца с занесением туда номера сегмента
4. Сложение всех групп в RFM данные

In [32]:
rfm_analysis

,code_module,id_student,R,F,M,r_quartile,f_quartile,m_quartile,RFM
0,CCC,23698,243.0,4.0,80.0,1,2,2,122
1,CCC,27116,243.0,5.0,96.0,1,1,1,111
2,CCC,28787,243.0,2.0,44.0,1,4,4,144
3,CCC,29420,244.0,3.5,62.0,1,2,3,123
4,CCC,29639,248.0,4.0,74.0,1,2,2,122
...,...,...,...,...,...,...,...,...,...
4384,DDD,2693243,230.0,4.5,82.0,4,1,2,412
4385,DDD,2694886,236.0,3.5,69.0,3,2,3,323
4386,DDD,2694933,230.0,4.0,73.0,4,2,2,422
4387,DDD,2695608,237.0,4.0,73.0,3,2,2,322
